# Interactive Analytics
<br>
## Task
Run interactive queries on the data and answer some analytical questions

## Data
* use two datasets about influencers
* the first dataset contains basic information about each influencer
* the second dataset contains posting history for each influncer for the past 6 months

## Questions:
1) Count number of distinct languages used in the dataset

2) List all interests in descending order depending on its frequency

3) How many influencers interested in Technology using english language (en) published a post in july and did not publish a post in june

4) For each influencer add a new column n_posts with number of posts he published in its history 

5) Find top 10 influencers that posted the most posts in their history

6) Compute entropy of the n_posts column for each interest

## Documentation
<br>
* Pyspark documentation of DataFrame API is <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html">here</a>

* Prezentation slides are accessed <a target="_blank" href = "https://docs.google.com/presentation/d/1XNKIfE5Atj_Mzse0wjmbwLecmVs2YkWm9cqOLqDVWPo/edit?usp=sharing">here</a>

### Import functions and modules

In [4]:
from pyspark.sql.functions import count, col, size, greatest, collect_list, when, explode, sum, rank, dense_rank, desc, row_number, avg, pandas_udf, PandasUDFType, array_contains, array_sort, lit, dayofweek
from pyspark.sql import Window
from pyspark.sql.types import DateType, DoubleType, StructType, StructField, DecimalType, LongType, StringType

import scipy.stats

### Load the data

In [6]:
infls = spark.table('mlprague.influencers')

posts_history = spark.table('mlprague.infl_posting_history')

### Explore the data

hint
* check the schema call df.printSchema()
* check the row count - call df.count()
* see some rows - call df.show(), or display(df)

In [8]:
infls.printSchema()

root
-- influencer_id: integer (nullable = true)
-- languages: array (nullable = true)
 |-- element: string (containsNull = true)
-- interests: array (nullable = true)
 |-- element: string (containsNull = true)

In [9]:
display(infls)

influencer_id,languages,interests
85194,List(en),"List(Family and relationships, Entertainment)"
67648,List(fr),List(Shopping and fashion)
184917,List(en),"List(Hobbies and activities, Food and drink, Family and relationships)"
370126,List(en),List(Hobbies and activities)
246388,List(es),"List(Sports and outdoors, Fitness and wellness)"
228457,List(en),"List(Entertainment, Hobbies and activities)"
122770,List(id),"List(Business and industry, Hobbies and activities)"
41916,List(ru),"List(Food and drink, Business and industry, Family and relationships)"
269152,List(hi),"List(Family and relationships, Hobbies and activities)"
204386,List(ru),List(Hobbies and activities)


In [10]:
infls.count()

Out[ 9 ]: 389553

### Count number of distinct languages used in the dataset (*)

hint
* use <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.explode">explode</a> and <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.distinct">distinct</a>

In [12]:
(
  infls
  .withColumn('language', explode('languages'))
  .select('language')
  .distinct()
  .count()
)

Out[ 10 ]: 91

### List all interests in descending order depending on its frequency (\* \*)

hint:
* use explode on the array
* use groupBy with some aggregation function (slide 25 in the presentation might be useful for aggregations)
* use <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.orderBy">orderBy</a>

In [14]:
display(
  infls
  .withColumn('interest', explode('interests'))
  .groupBy('interest')
  .agg(
    count('*').alias('interest_frequency')
  )
  .orderBy(desc('interest_frequency'))
)

interest,interest_frequency
Hobbies and activities,197725
Shopping and fashion,122291
Family and relationships,106460
Entertainment,80037
Business and industry,79194
Food and drink,50671
Fitness and wellness,39184
Sports and outdoors,28621
Technology,10751


### How many influencers interested in Technology using english language (en) published a post in july and did not publish a post in june (\* \* \*)

hint
* Possible way how to do it:
 * join influencers with their posting history
 * create 2 dataframes - in the first one select influencers that published in july, in the second select influencers that published in june
 * apply filters for interest and language
 * do a left anti join ('left_anti')
 * deduplicate the result
* <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.join">joins</a> in the docs
* slide 20 in the presentation might be useful for joins
* <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.dropDuplicates">dropDuplicates</a> in the docs

In [16]:
july = (
  infls
  .join(posts_history, 'influencer_id')
  .filter(array_contains(col('languages'), 'en'))
  .filter(array_contains(col('interests'), 'Technology'))
  .filter(col('post_date').between('2018-07-01', '2018-07-31'))
)

june = (
  infls
  .join(posts_history, 'influencer_id')
  .filter(array_contains(col('languages'), 'en'))
  .filter(array_contains(col('interests'), 'Technology'))
  .filter(col('post_date').between('2018-06-01', '2018-06-30'))
)

july.join(june, 'influencer_id', 'left_anti').dropDuplicates(['influencer_id']).count()


Out[ 12 ]: 178

### For each influencer add a new column n_posts with number of posts he published in its history (\* \*)

hint
* This question is prerequisite for the two next questions
* Possible way how to do it:
 * aggregate the posting history and join the result to influencers

In [18]:
n_posts = (
  posts_history
  .groupBy('influencer_id')
  .agg(
    count('*').alias('n_posts')
  )
)

infls_with_n_posts = (
  infls
  .join(n_posts, 'influencer_id')
  
)

In [19]:
display(infls_with_n_posts)

influencer_id,languages,interests,n_posts
225306,List(pt),List(Hobbies and activities),57
207398,List(pt),List(Hobbies and activities),129
120861,List(en),"List(Entertainment, Business and industry, Hobbies and activities)",40
184976,List(en),"List(Shopping and fashion, Fitness and wellness, Hobbies and activities)",154
333626,List(en),"List(Hobbies and activities, Family and relationships, Business and industry)",175
146036,List(es),"List(Entertainment, Hobbies and activities, Family and relationships)",162
175634,List(pt),List(Hobbies and activities),23
350570,List(tr),"List(Shopping and fashion, Business and industry)",81
82529,List(en),List(Family and relationships),36
23015,List(en),List(Shopping and fashion),122


### Find top 10 influencers that posted the most posts in their history (*)

hint
* this question depends on the result from the previous question
* orderBy the previous result

In [21]:
display(
  infls_with_n_posts
  .orderBy(desc('n_posts'))
  .limit(10)
)

influencer_id,languages,interests,n_posts
330857,List(pt),"List(Entertainment, Technology)",5893
196937,List(en),"List(Entertainment, Family and relationships, Hobbies and activities)",4118
191706,List(en),"List(Technology, Entertainment, Family and relationships)",3774
329997,List(pt),"List(Hobbies and activities, Family and relationships, Technology)",3491
8865,List(en),"List(Business and industry, Entertainment, Family and relationships)",3298
48810,List(ru),List(Shopping and fashion),3179
360858,List(en),"List(Entertainment, Business and industry, Fitness and wellness)",3011
320308,"List(en, pt)","List(Entertainment, Business and industry, Hobbies and activities)",2926
340631,List(en),"List(Hobbies and activities, Family and relationships)",2855
336677,List(pt),List(Family and relationships),2851


### Compute entropy of the n_posts column for each interest (\* \* \* \* \*)

hint
* This question depends on results from the previous 2 questions
* There are at least 2 ways how you can solve this (choose which way you prefer)
* Way 1:
 * Using higer order functions (slide 55 in the presentation might be useful)
 * first compute probability for each interest (for each interest get an array with probabilities of n_posts col)
 * (you might want to groupBy interest and n_posts than use a window and finaly groupBy again by interest and use collect_list for final aggregation)
 * use AGGREGATE in SQL expression to compute the entropy from the array
* Way 2:
  * Using Pandas grouped_map UDF (slide 40 in the presentation might be useful)
  * Define schema for the udf
  * Implement the udf (to compute the probabilities you can use value_counts() of Pandas.Series and divide by number of rows using len(pandas dataframe))
  * Use scipy.stats.entropy for the entropy calculation
  
Note
* For entropy calculation use the same definition as is used in scipy: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.entropy.html

$$S = - \Sigma_{i} \left(p\left(i\right) * \log\left(p\left(i\right)\right)\right)$$

* for probability computation you can use this approach: compute the frequency of each value and divide by the total number of values
 * example: suppose we have this array [10, 10, 11, 12], the probability of 10 is p(10) = 2/4 = 0.5, the probability of 11 is p(11) = 1/4 = 0.25

In [23]:
# Using Higher order functions:

interests_with_probability = (
  infls_with_n_posts
  .select('n_posts', explode('interests').alias('interest'))
  .select(col('n_posts').cast(DoubleType()).alias('n_posts'), 'interest')
  .groupBy('interest', 'n_posts')
  .agg(count('*').alias('frequency'))
  .withColumn('n_posts_total', sum('frequency').over(Window.partitionBy('interest')))
  .withColumn('n_posts_probability', col('frequency') / col('n_posts_total'))
  .groupBy('interest')
  .agg(
    collect_list(col('n_posts_probability')).alias('n_posts')
  )
)

In [24]:
display(interests_with_probability)

interest n_posts Business and industry List(0.0024117988736520445, 0.01006389372932293, 0.009470414425335253, 7.828875924943809E-4, 8.58650907897063E-4, 2.5254438467560673E-5, 1.1364497310402303E-4, 0.004040710154809708, 5.0508876935121346E-5, 1.1364497310402303E-4, 2.5254438467560673E-5, 2.5254438467560673E-5, 1.2627219233780336E-5, 1.2627219233780336E-5, 1.2627219233780336E-5, 1.2627219233780336E-5, 8.207692501957219E-4, 1.2627219233780336E-5, 1.2627219233780337E-4, 2.9042604237694775E-4, 1.2627219233780336E-5, 3.788165770134101E-5, 3.788165770134101E-5, 0.006465136247695532, 1.388994115715837E-4, 2.0203550774048538E-4, 8.839053463646235E-5, 8.839053463646235E-5, 6.566154001565775E-4, 3.2830770007828876E-4, 1.2627219233780336E-5, 1.0101775387024269E-4, 2.5254438467560673E-5, 0.009470414425335253, 0.005694875874434932, 1.2627219233780336E-5, 0.009457787206101473, 4.166982347147511E-4, 1.2627219233780336E-5, 5.0508876935121346E-5, 2.1466272697426574E-4, 7.197514963254792E-4, 6.313609616890169E-5, 6.313609616890169E-5, 7.576331540268203E-5, 1.2627219233780336E-5, 0.006654544536202237, 5.0508876935121346E-5, 2.0203550774048538E-4, 1.2627219233780336E-5, 2.5254438467560673E-5, 1.2627219233780336E-5, 2.5254438467560673E-5, 7.071242770916988E-4, 2.2728994620804606E-4, 0.007399550470995277, 6.313609616890169E-5, 2.5254438467560673E-5, 0.0052276687627850595, 0.0047983433088365285, 1.2627219233780336E-5, 5.934793039876758E-4, 1.5152663080536405E-4, 1.2627219233780336E-5, 1.2627219233780336E-5, 2.5254438467560673E-5, 0.010556355279440361, 8.839053463646235E-5, 2.0203550774048538E-4, 2.5254438467560673E-5, 1.2627219233780336E-5, 0.005808520847538955, 0.0021213728312750968, 2.9042604237694775E-4, 1.2627219233780336E-5, 2.5254438467560673E-5, 0.007083869990150769, 0.005758011970603834, 1.388994115715837E-4, 8.839053463646235E-5, 1.2627219233780336E-5, 0.00863701795590575, 5.934793039876758E-4, 1.2627219233780336E-5, 0.005732757532136273, 2.5254438467560673E-5, 1.2627219233780336E-5, 1.2627219233780336E-5, 2.5254438467560673E-5, 0.007828875924943808, 0.001313230800313155, 6.313609616890169E-5, 1.6415385003914438E-4, 2.5254438467560673E-5, 2.5254438467560673E-5, 0.009419905548400132, 6.313609616890169E-5, 1.2627219233780336E-5, 3.788165770134101E-5, 1.2627219233780336E-5, 3.788165770134101E-5, 2.5254438467560673E-5, 7.576331540268203E-5, 1.2627219233780336E-5, 1.2627219233780336E-5, 1.2627219233780336E-5, 1.2627219233780336E-5, 2.5254438467560673E-5, 0.006780816728540041, 0.008346591913528803, 1.2627219233780336E-5, 1.2627219233780336E-5, 3.788165770134101E-5, 1.2627219233780336E-5, 1.2627219233780336E-5, 1.2627219233780336E-5, 3.788165770134101E-4, 5.0508876935121346E-5, 3.788165770134101E-5, 1.2627219233780336E-5, 0.004015455716342147, 5.0508876935121346E-5, 3.2830770007828876E-4, 2.5254438467560673E-5, 5.682248655201152E-4, 1.2627219233780336E-5, 1.2627219233780336E-5, 0.005707503093668713, 0.002727479354496553, 1.2627219233780337E-4, 1.2627219233780336E-5, 0.005846402505240296, 1.2627219233780336E-5, 1.2627219233780336E-5, 1.2627219233780336E-5, 0.0055938581205646895, 7.576331540268203E-5, 1.2627219233780336E-5, 0.002373917215950703, 8.839053463646235E-5, 1.2627219233780336E-5, 0.007563704321034422, 3.788165770134101E-5, 1.2627219233780336E-5, 0.009672449933075738, 1.1364497310402303E-4, 2.5254438467560673E-5, 0.0038639290855367833, 1.2627219233780336E-5, 0.008409728009697704, 5.808520847538955E-4, 6.313609616890169E-5, 8.081420309619415E-4, 1.2627219233780336E-5, 0.004836224966537869, 3.6618935777962976E-4, 3.788165770134101E-5, 1.2627219233780336E-5, 0.005429704270525545, 0.002424426092885825, 1.2627219233780336E-5, 0.008864307902113797, 0.0014395029926509584, 0.00936939667146501, 6.313609616890169E-5, 3.788165770134101E-5, 1.2627219233780336E-5, 1.2627219233780336E-5, 2.5254438467560673E-5, 2.5254438467560673E-5, 9.217870040659646E-4, 1.2627219233780336E-5, 3.788165770134101E-5, 2.5254438467560673E-5, 1.2627219233780336E-5, 2.5254438

In [25]:
result = interests_with_probability.selectExpr(
  'interest',
  'AGGREGATE(n_posts, CAST(0 AS double), (buffer, value) -> (buffer - value * Log(value))) AS n_posts_entropy'
).orderBy('n_posts_entropy')

In [26]:
display(result)

interest,n_posts_entropy
Hobbies and activities,5.117559768873704
Fitness and wellness,5.188415145903004
Sports and outdoors,5.1930014790478225
Shopping and fashion,5.222996850662198
Business and industry,5.241936901602908
Entertainment,5.248161952520844
Family and relationships,5.2524384118387095
Food and drink,5.30943550418276
Technology,5.354048021474125


In [27]:
# using Pandas GROUPED_MAP UDF:

interests_with_n_posts = (
  infls_with_n_posts
  .select('n_posts', explode('interests').alias('interest'))
)


schema = StructType(
  [
    StructField('n_posts', LongType()),
    StructField('interest', StringType()),
    StructField('entropy', DoubleType())
  ]
)

@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
def udf_entropy(pdf):
    p_data= pdf['n_posts'].value_counts() / len(pdf) # calculates the probabilities
    pdf['entropy'] = scipy.stats.entropy(p_data)
    return pdf

In [28]:
result = (
  interests_with_n_posts
  .groupBy('interest')
  .apply(udf_entropy)
  .select('interest', 'entropy')
  .dropDuplicates()
  .orderBy('entropy')
)

In [29]:
display(result)

interest,entropy
Hobbies and activities,5.117559768873676
Fitness and wellness,5.188415145903016
Sports and outdoors,5.193001479047821
Shopping and fashion,5.22299685066222
Business and industry,5.241936901602921
Entertainment,5.248161952520835
Family and relationships,5.2524384118387415
Food and drink,5.309435504182742
Technology,5.35404802147411


### You may want to take a look at the distribution for given interests

In [31]:
display(
  interests_with_n_posts
  .filter(col('interest') == 'Technology')
  .groupBy('n_posts')
  .agg(
    count('*')
  )
  .orderBy('n_posts')
)

n_posts,count(1)
11,30
12,38
13,49
14,48
15,44
16,58
17,41
18,54
19,58
20,70


In [32]:
display(
  interests_with_n_posts
  .filter(col('interest') == 'Hobbies and activities')
  .groupBy('n_posts')
  .agg(
    count('*')
  )
  .orderBy('n_posts')
)

n_posts,count(1)
11,1106
12,1216
13,1269
14,1422
15,1425
16,1474
17,1505
18,1683
19,1674
20,1706
